In [ ]:
# Import libraries

import os
import re
from docx import Document
from docx.oxml.ns import qn
from docx.shared import Pt, Inches
from docx.oxml import OxmlElement
import pandas as pd
import random
from docx.enum.table import WD_ROW_HEIGHT_RULE

#### Read In Template and Questions

In [2]:
# Define folder path relative to the current script or working directory
folder_path = os.path.join(".", "Docs")

# Construct full file paths
# Card Template
word_file = os.path.join(folder_path, "Birthday-Bingo-Cards.docx")
# Tasks/ Questions
excel_file = os.path.join(folder_path, "questions2.xlsx")

# Read Word document (Card Template)
try:
    doc = Document(word_file)
    print("Contents of the Word document:")
    for para in doc.paragraphs:
        print(para.text)
except Exception as e:
    print(f"Error reading Word document: {e}")

# Read Excel file (Tasks/ Questions)
try:
    df = pd.read_excel(excel_file)
    df.columns = ['Sätze']
    print("\nContents of the Excel fifole:")
    print(df.head())  # Show first few 
    
    

except Exception as e:
    print(f"Error reading Excel file: {e}")

Contents of the Word document:
birthday-Bingo



Contents of the Excel fifole:
                                         Sätze
0                                Ist Oma/ Opa:
1  Hat am selben Tag wie ein Promi Geburtstag:
2                  Hat in einem Chor gesungen:
3   War bei Goldhochzeit der Großeltern dabei:
4                          Besitzt einen Hund:


In [3]:
df.size

90

#### Random Selection of Questions

In [4]:
# 3. Random selection of 25 sentences out of dataframe
selected_sentences = random.sample(df['Sätze'].tolist(), 25)

print(selected_sentences)

['Kennt Klaus länger als 40 Jahre:', 'Hat schon bei einem Konzert mitgewirkt:', 'Hat als Kind gerne Comics gelesen:', 'Ist schon mal bei einer Prüfung durchgefallen:', 'Hat dieses Jahr einen Strafzettel bekommen:', 'Trinkt lieber Tee als Kaffee:', 'War bei einem Pfadfinderlager:', 'War mal auf einem Kreuzfahrtschiff:', 'Ist mit Klaus verwandt:', 'War schon im Fernsehen zu sehen:', 'Hat ein Auto älter als 10 Jahre:', 'Spielt gern Golf:', 'Hatte schon einen Knochenbruch:', 'Isst gerne Fisch:', 'War schon auf einem Musikfestival:', 'Isst kein Fleisch:', 'Hat ein Gemüsebeet:', 'Hat alle runden Geburtstage groß gefeiert:', 'War mit Klaus schon im Urlaub:', 'Kann ohne sein Smartphone nicht leben:', 'Hat schon einen Promi getroffen:', 'Mehr als 20 km angereist:', 'Hat im März Geburtstag:', 'Hat in einem Chor gesungen:', 'Ist einen Marathon/ Halbmarathon gelaufen:']


#### Edit word template

In [ ]:
import random
from docx import Document
from docx.shared import Pt, Inches
from docx.enum.table import WD_ROW_HEIGHT_RULE

# Random selection of 25 sentences
selected_sentences = random.sample(df['Sätze'].tolist(), 25)
print(selected_sentences)

# Load the Word template
table = doc.tables[0]

# --- Set page margins (narrow) ---
for section in doc.sections:
    section.bottom_margin = Inches(0.2)
    # Optional: Uncomment for narrower layout
    # section.top_margin = Inches(0.4)
    # section.left_margin = Inches(0.4)
    # section.right_margin = Inches(0)

# --- Estimate font size based on longest sentence ---
max_len = max(len(s) for s in selected_sentences)
if max_len > 50:
    font_size = Pt(8)
elif max_len > 35:
    font_size = Pt(9)
else:
    font_size = Pt(10)


def fill_table_with_font(table):
    index = 0
    for row in table.rows:
        row.height_rule = WD_ROW_HEIGHT_RULE.AT_LEAST
        row.height = Inches(0.5)
        for cell in row.cells:
            cell.text = ""
            para = cell.paragraphs[0]

            sentence = selected_sentences[index]
            font_size_pt = 9 if len(sentence) > 35 else 10

            run = para.add_run(sentence)
            run.font.size = Pt(font_size_pt)

            # Add spare line
            cell.add_paragraph()
            index += 1




def adjust_column_widths(table, total_width_inch=6.0):
    num_cols = len(table.columns)
    col_width = Inches(total_width_inch / num_cols)
    for row in table.rows:
        for cell in row.cells:
            cell.width = col_width


# --- Fill and adjust ---
#fill_table_with_font(table, font_size.pt)
fill_table_with_font(table)
adjust_column_widths(table)

# --- Fit check ---
estimated_total_height = len(table.rows) * 0.5 + 1.0
if estimated_total_height > 10.5:
    print("⚠️ Content may not fit. Adjusting font and column width...")
    font_size = Pt(8)
    fill_table_with_font(table, font_size.pt)
    adjust_column_widths(table, total_width_inch=5.5)

adjust_column_widths(table, total_width_inch=6.0)  # Force-fit

# --- Save document ---
doc.save("exp1.docx")

['Trinkt gerne Wein:', 'Hat dieses Jahr einen Strafzettel bekommen:', 'Macht eine ehrenamtliche Tätigkeit:', 'Hat Bungee-Jumping gemacht:', 'Hat als Kind gerne Comics gelesen:', 'Mag Krimis:', 'Hat bei einem Gewinnspiel gewonnen:', 'Hat ein Gemüsebeet:', 'Fährt gerne Fahrrad:', 'Hat ein/e Lieblingsschauspieler/in:', 'War mit Klaus schon im Urlaub:', 'Mehr als 50 km angereist', 'Kennt Klaus seit der Schulzeit:', 'Mehr als 20 km angereist:', 'War im letzten Jahr bei einem Konzert:', 'Kann häkeln:', 'War schon bei der Polizei:', 'Hat Angst vor Spinnen:', 'War bei Goldhochzeit der Großeltern dabei:', 'Liest regelmäßig Zeitung:', 'Hat ein Tatoo:', 'Liest gerade ein Buch:', 'Hat schon bei einem Konzert mitgewirkt:', 'Hat ein Haustier:', 'Wurde schon von einem Tier gebissen:']


In [78]:
sentence_pool = df['Sätze'].dropna().tolist()

# === Loop to generate 50 documents ===
for i in range(1, 51):
    # 1. Select 25 random sentences
    selected_sentences = random.sample(sentence_pool, 25)

    # 2. Load template doc
    #doc = Document("YourTemplate.docx")  # Replace with actual path
    table = doc.tables[0]

    # 3. Set page margins (narrow)
    for section in doc.sections:
        section.bottom_margin = Inches(0.2)
        # Optionally uncomment other margins
        # section.top_margin = Inches(0.4)
        # section.left_margin = Inches(0.4)
        # section.right_margin = Inches(0)

    # 4. Determine suitable font size
    max_len = max(len(s) for s in selected_sentences)
    if max_len > 70:
        font_size = Pt(8)
    elif max_len > 50:
        font_size = Pt(9)
    else:
        font_size = Pt(10)

    # 5. Fill and format table
    fill_table_with_font(table)
    adjust_column_widths(table)

    # 6. Heuristic layout check
    estimated_total_height = len(table.rows) * 0.5 + 1.0
    if estimated_total_height > 10.5:
        font_size = Pt(8)
        fill_table_with_font(table, selected_sentences, font_size.pt)
        adjust_column_widths(table, total_width_inch=5.5)

    # 7. Final column width safeguard
    adjust_column_widths(table, total_width_inch=6.0)

    # 8. Save with dynamic filename
    filename = f"exp{i}.docx"
    doc.save(filename)
    print(f"✅ Created {filename}")


✅ Created exp1.docx
✅ Created exp2.docx
✅ Created exp3.docx
✅ Created exp4.docx
✅ Created exp5.docx
✅ Created exp6.docx
✅ Created exp7.docx
✅ Created exp8.docx
✅ Created exp9.docx
✅ Created exp10.docx
✅ Created exp11.docx
✅ Created exp12.docx
✅ Created exp13.docx
✅ Created exp14.docx
✅ Created exp15.docx
✅ Created exp16.docx
✅ Created exp17.docx
✅ Created exp18.docx
✅ Created exp19.docx
✅ Created exp20.docx
✅ Created exp21.docx
✅ Created exp22.docx
✅ Created exp23.docx
✅ Created exp24.docx
✅ Created exp25.docx
✅ Created exp26.docx
✅ Created exp27.docx
✅ Created exp28.docx
✅ Created exp29.docx
✅ Created exp30.docx
✅ Created exp31.docx
✅ Created exp32.docx
✅ Created exp33.docx
✅ Created exp34.docx
✅ Created exp35.docx
✅ Created exp36.docx
✅ Created exp37.docx
✅ Created exp38.docx
✅ Created exp39.docx
✅ Created exp40.docx
✅ Created exp41.docx
✅ Created exp42.docx
✅ Created exp43.docx
✅ Created exp44.docx
✅ Created exp45.docx
✅ Created exp46.docx
✅ Created exp47.docx
✅ Created exp48.docx
✅

#### Merge single files to one

In [ ]:
import os
from docx import Document

def merge_word_documents(output_filename, num_files):
    #base_path = os.path.dirname(os.path.abspath(__file__))
    #print(base_path)
    base_path = os.path.join(".", "Docs")
    
    merged_doc = None

    for i in range(1, num_files + 1):
        filename = os.path.join(base_path, f"exp{i}.docx")
        if not os.path.exists(filename):
            print(f"⚠️ Datei {filename} nicht gefunden, wird übersprungen.")
            continue
        
        print(f"Lade {filename} ...")
        doc = Document(filename)

        if merged_doc is None:
            merged_doc = Document(filename)  # Start with first document
        else:
            # Append contents of current document
            for element in doc.element.body:
                merged_doc.element.body.append(element)

    if merged_doc is not None:
        output_path = os.path.join(base_path, output_filename)
        merged_doc.save(output_path)
        print(f"✅ Dokument erfolgreich gespeichert unter: {output_path}")
    else:
        print("Keine Dokumente zum Zusammenfügen gefunden.")


folder_path = os.path.join(".", "Docs")
word_file = os.path.join(folder_path, "Spielkarten.docx")



merge_word_documents("Spielkarten.docx", 50)


Lade .\Docs\exp1.docx ...
Lade .\Docs\exp2.docx ...
Lade .\Docs\exp3.docx ...
Lade .\Docs\exp4.docx ...
Lade .\Docs\exp5.docx ...
Lade .\Docs\exp6.docx ...
Lade .\Docs\exp7.docx ...
Lade .\Docs\exp8.docx ...
Lade .\Docs\exp9.docx ...
Lade .\Docs\exp10.docx ...
Lade .\Docs\exp11.docx ...
Lade .\Docs\exp12.docx ...
Lade .\Docs\exp13.docx ...
Lade .\Docs\exp14.docx ...
Lade .\Docs\exp15.docx ...
Lade .\Docs\exp16.docx ...
Lade .\Docs\exp17.docx ...
Lade .\Docs\exp18.docx ...
Lade .\Docs\exp19.docx ...
Lade .\Docs\exp20.docx ...
Lade .\Docs\exp21.docx ...
Lade .\Docs\exp22.docx ...
Lade .\Docs\exp23.docx ...
Lade .\Docs\exp24.docx ...
Lade .\Docs\exp25.docx ...
Lade .\Docs\exp26.docx ...
Lade .\Docs\exp27.docx ...
Lade .\Docs\exp28.docx ...
Lade .\Docs\exp29.docx ...
Lade .\Docs\exp30.docx ...
Lade .\Docs\exp31.docx ...
Lade .\Docs\exp32.docx ...
Lade .\Docs\exp33.docx ...
Lade .\Docs\exp34.docx ...
Lade .\Docs\exp35.docx ...
Lade .\Docs\exp36.docx ...
Lade .\Docs\exp37.docx ...
Lade .\Doc